In [1]:
%matplotlib inline
import numpy as np
import sys
sys.path.append("build-clang-5.0/language_bindings/python")
import muSpectre as msp
import matplotlib.pyplot as plt



In [5]:
## currently, muSpectre is restricted to odd-numbered resolutions for
## reasons explained in T.W.J. de Geus, J. Vondřejc, J. Zeman,
## R.H.J. Peerlings, M.G.D. Geers, Finite strain FFT-based non-linear
## solvers made simple, Computer Methods in Applied Mechanics and
## Engineering, Volume 318, 2017
## https://doi.org/10.1016/j.cma.2016.12.032
resolution = [51, 51]
center = np.array([r//2 for r in resolution])
incl = resolution[0]//5


## Domain dimensions
lengths = [7., 5.]
## formulation (small_strain or finite_strain)
formulation = msp.Formulation.finite_strain

## build a computational domain
rve = msp.Cell(resolution, lengths, formulation)

## define the material properties of the matrix and inclusion
hard = msp.material.MaterialHyperElastoPlastic1_2d.make(
    rve, "hard", 10e9, .33, .006e9,  h=.01e9)
#hard = msp.material.MaterialLinearElastic1_2d.make(
#    rve, "hard", 10e9, .33)
soft = msp.material.MaterialLinearElastic1_2d.make(
    rve, "soft",  70e9, .33)

## assign each pixel to exactly one material
for i, pixel in enumerate(rve):
    if np.linalg.norm(center - np.array(pixel),2)<incl:
        hard.add_pixel(pixel)
    else:
        soft.add_pixel(pixel)

## define the convergence tolerance for the Newton-Raphson increment
tol = 1e-5
## tolerance for the solver of the linear cell
cg_tol = 1e-6
eq_tol = 1e-8


## Macroscopic strain
def get_Del0(gammas):
    def Del0(gamma):
        Del0 = np.array([[.0, gamma],
                         [0,  0.]])
        return Del0#.5*(Del0 + Del0.T)
    return [Del0(gamma) for gamma in gammas]

del_gamma = 1e-3 # increments
Del_gamma = 3e-2 # total strain
nb_gamma = int(Del_gamma/del_gamma)
Del0 = get_Del0((i*del_gamma for i in range(1, nb_gamma+1)))


maxiter = 1000 ## for linear cell solver

## Choose a solver for the linear cells. Currently avaliable:
## SolverCG, SolverCGEigen, SolverBiCGSTABEigen, SolverGMRESEigen,
## SolverDGMRESEigen, SolverMINRESEigen.
## See Reference for explanations
solver = msp.solvers.SolverCG(rve, cg_tol, maxiter, verbose=False)


## Verbosity levels:
## 0: silent,
## 1: info about Newton-Raphson loop,
verbose = 1

## Choose a solution strategy. Currently available:
## de_geus: is described in de Geus et al. see Ref above
## newton_cg: classical Newton-Conjugate Gradient solver. Recommended
result = msp.solvers.newton_cg(rve, Del0, solver, tol, eq_tol, verbose=verbose)


RuntimeError: Failure at load step 3 of 30. In Newton-Raphson step 4:
Conjugate gradient has not converged. After 5801 steps, the solver  FAILED with  |r|/|b| =            -nan, cg_tol = 1e-06

The applied boundary condition is F =
    0 0.003
    0     0
and the load increment is ΔF =
    0 0.001
    0     0


In [ ]:
mean_stress = [res.stress.reshape(-1, 4).mean(axis=0) for res in result]

mean_tau = np.array([stress[2] for stress in mean_stress])
gammas = np.array([del0[0,1] for del0 in Del0])
plt.plot(gammas, mean_tau)
